In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

data = pd.read_csv("DATA/Minmax_Scaler_result.csv",encoding="cp949")
data.head()

,CUSTNO,GNO,CBSCORE,CBSCOREGRD,CREDITOTAMT,YSALEAMT,ESTMM,ASSETAMT,IMSAAMT,IMJUAMT,...,환율,GDP,소상공인체감지수,실업률,물가지수,국고채,금리,유가등락률,소비자심리지수,ONEHOT
0,475821,l180202101898,0.949290,0.000000,0.002338,0.223956,0.058085,0.000768,0.004153,0.000000,...,0.436398,0.989382,0.320660,0.346154,0.928674,0.144172,0.000000,0.511688,0.956072,3
1,74417,l230201700120,0.897566,0.166667,0.002221,0.129121,0.014129,0.001920,0.000000,0.015489,...,0.518949,0.691828,0.355093,0.423077,0.707266,0.257669,0.272727,0.423701,0.545220,8
2,387787,l110201603233,0.909736,0.000000,0.000000,0.083846,0.290424,0.000768,0.004153,0.000000,...,0.587992,0.661863,0.624821,0.346154,0.634454,0.239264,0.272727,0.289610,0.578811,7
3,395418,l230201700254,0.884381,0.166667,0.001637,0.079560,0.017268,0.000768,0.004153,0.000000,...,0.516323,0.691828,0.676471,0.307692,0.703799,0.269939,0.272727,0.332792,0.604651,3
4,190372,l200201601418,0.726166,0.833333,0.000818,0.036813,0.262166,0.006143,0.000831,0.000000,...,0.504690,0.643228,0.489957,0.307692,0.592352,0.119632,0.272727,0.190909,0.715762,7


In [55]:
features = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUSAAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'LABORCNT', 'KOSPI', '환율', 'GDP', 
       '소상공인체감지수', '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','ONEHOT']]
act = data['ACTCD']

In [56]:
act = act - 1
act = act * -1

In [57]:
from sklearn.model_selection import train_test_split

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 22) (11736, 22) (27381,) (11736,)


In [58]:
from imblearn.over_sampling import BorderlineSMOTE, ADASYN
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
from imblearn.combine import SMOTEENN, SMOTETomek

#언더 샘플링
enn = EditedNearestNeighbours(kind_sel="all", n_neighbors=10)
tomekl = TomekLinks()

#오버 샘플링
bsmote = BorderlineSMOTE(random_state=42)
adasyn = ADASYN(random_state=42)

#혼합 샘플링
smotee = SMOTEENN(random_state=42)
smoteT = SMOTETomek(random_state=42)


X_under1_train, Y_under1_train = enn.fit_resample(train_x, train_y)
X_under2_train, Y_under2_train = tomekl.fit_resample(train_x, train_y)

X_over1_train, Y_over1_train = bsmote.fit_resample(train_x,train_y)
X_over2_train, Y_over2_train = adasyn.fit_resample(train_x,train_y)

X_comb1_train, Y_comb1_train = smotee.fit_resample(train_x, train_y)
X_comb2_train, Y_comb2_train = smoteT.fit_resample(train_x, train_y)

# 오버샘플링

1. BorderlineSMOTE

In [23]:
def cut_off(y, threshold) :
    Y = y.copy()  # 대문자 Y를 새로운 변수로 하여 기존의 y값에 영향이 가지 않도록 한다.
    Y[Y>threshold] = 1
    Y[Y<threshold] = 0
    return Y.astype(int)

from sklearn.metrics import confusion_matrix
# confusion matrix accuracy(정확도) 계산함수

def acc(cfmat):
    return (cfmat[0,0] + cfmat[1,1])/(cfmat[0,0] + cfmat[1,1] + cfmat[0,1] + cfmat[1,0])

In [24]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [25]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)

In [8]:
from sklearn.model_selection import cross_val_score
cross_val_score(eclf, X_over1_train, Y_over1_train, cv=5).mean()

0.8544504721719912

In [26]:
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier()

DecisionTreeClassifier()

In [27]:
AdaBoostClassifier()

AdaBoostClassifier()

In [28]:
params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

In [29]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_over1_train, Y_over1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.55,
 'n_estimators': 27}

In [13]:
grid.best_estimator_.feature_importances_

array([0.07766098, 0.04684005, 0.02373662, 0.08715234, 0.07228255,
       0.07062534, 0.01287007, 0.0402632 , 0.08371823, 0.13837393,
       0.11922424, 0.        , 0.00126807, 0.04223189, 0.00599657,
       0.00151928, 0.02532738, 0.02047163, 0.0243234 , 0.        ,
       0.00419939, 0.10191486])

In [59]:
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score

dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.55, n_estimators = 27)
model = abc.fit(X_over1_train, Y_over1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [60]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[ 564,  506],
       [2339, 8327]], dtype=int64)

In [61]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.19      0.53      0.28      1070
        정상보증       0.94      0.78      0.85     10666

    accuracy                           0.76     11736
   macro avg       0.57      0.65      0.57     11736
weighted avg       0.87      0.76      0.80     11736



In [32]:
accuracy = acc(cfmat)
accuracy

0.7515337423312883

In [16]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.1796707229778096
recall:  0.4691588785046729
f1 score:  0.25983436853002073
accuracy:  0.6271352239888826


2. ADASYN

In [17]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_over2_train, Y_over2_train, cv=5).mean()

0.8688921613678321

In [34]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_over2_train, Y_over2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 7,
 'learning_rate': 0.6,
 'n_estimators': 27}

In [62]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth = 2, max_features = 7)
abc = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 27)
model = abc.fit(X_over2_train, Y_over2_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [64]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[ 515,  555],
       [2365, 8301]], dtype=int64)

In [63]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.18      0.48      0.26      1070
        정상보증       0.94      0.78      0.85     10666

    accuracy                           0.75     11736
   macro avg       0.56      0.63      0.56     11736
weighted avg       0.87      0.75      0.80     11736



In [21]:
accuracy = acc(cfmat)
accuracy

0.7442058623040219

In [22]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.16176470588235295
recall:  0.43177570093457945
f1 score:  0.23535404992358636
accuracy:  0.6036620863570329


# 언더샘플링

1. EditedNearestNeighbours

In [23]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_under1_train, Y_under1_train, cv=5).mean()

0.874166878819404

In [38]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_under1_train, Y_under1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.45,
 'n_estimators': 24}

In [65]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.45, n_estimators = 24)
model = abc.fit(X_under1_train, Y_under1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [66]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[ 297,  773],
       [ 866, 9800]], dtype=int64)

In [67]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.26      0.28      0.27      1070
        정상보증       0.93      0.92      0.92     10666

    accuracy                           0.86     11736
   macro avg       0.59      0.60      0.59     11736
weighted avg       0.87      0.86      0.86     11736



In [28]:
accuracy = acc(cfmat)
accuracy

0.8611111111111112

In [29]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.2719869706840391
recall:  0.3121495327102804
f1 score:  0.2906875543951262
accuracy:  0.6141658970508086


2. TomekLinks

In [30]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_under2_train, Y_under2_train, cv=5).mean()

0.9063297857647399

In [42]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_under2_train, Y_under2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.55,
 'n_estimators': 25}

In [68]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.55, n_estimators = 25)
model = abc.fit(X_under2_train, Y_under2_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [69]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[    0,  1070],
       [    0, 10666]], dtype=int64)

In [70]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.00      0.00      0.00      1070
        정상보증       0.91      1.00      0.95     10666

    accuracy                           0.91     11736
   macro avg       0.45      0.50      0.48     11736
weighted avg       0.83      0.91      0.87     11736



C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
accuracy = acc(cfmat)
accuracy

0.9089127471029311

In [35]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  1.0
recall:  0.0009345794392523365
f1 score:  0.0018674136321195146
accuracy:  0.5004672897196262


# 혼합샘플링

1. SMOTEENN

In [36]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_comb1_train, Y_comb1_train, cv=5).mean()

0.875260659094

In [46]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_comb1_train, Y_comb1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.55,
 'n_estimators': 27}

In [71]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.55, n_estimators = 27)
model = abc.fit(X_comb1_train, Y_comb1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [72]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[ 760,  310],
       [3780, 6886]], dtype=int64)

In [73]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.17      0.71      0.27      1070
        정상보증       0.96      0.65      0.77     10666

    accuracy                           0.65     11736
   macro avg       0.56      0.68      0.52     11736
weighted avg       0.88      0.65      0.73     11736



In [40]:
accuracy = acc(cfmat)
accuracy

0.6435753237900477

In [41]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.15768638662854628
recall:  0.6700934579439253
f1 score:  0.25529642157735444
accuracy:  0.6555042575674999


2. SMOTETomek

In [42]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_comb2_train, Y_comb2_train, cv=5).mean()

0.863866503038396

In [50]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_comb2_train, Y_comb2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 27}

In [74]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 27)
model = abc.fit(X_comb1_train, Y_comb1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [75]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[ 741,  329],
       [3675, 6991]], dtype=int64)

In [76]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.17      0.69      0.27      1070
        정상보증       0.96      0.66      0.78     10666

    accuracy                           0.66     11736
   macro avg       0.56      0.67      0.52     11736
weighted avg       0.88      0.66      0.73     11736



In [46]:
accuracy = acc(cfmat)
accuracy

0.6453646898432175

In [47]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.15811587793011941
recall:  0.6682242990654206
f1 score:  0.25572246065808296
accuracy:  0.6556478705152717
